# クラス

In [1]:
#include <iostream>
#include <string>
#include <cmath>

using namespace std;

ここでは,一般的な方法であるヘッダファイルを用いたクラス定義をしている。ヘッダファイルでクラスの内容を定義し,その関数やデータの実装をソースファイルで行う。  
本来わざわざ定義と実装を分離する必要はない。しかし,特に大規模開発などでは共同作業を進めやすくするために分離することが多いのだとか。或いは,ライブラリなどでは実装内容はバイナリにして,ヘッダは公開することで提供される関数やオブジェクトを明示することができるか。

## クラス定義1 (Vector)

In [2]:
class Vector {

	public:
		// クラス変数
		const static string VERSION;
			/*
				クラス内で共有される値 (static)
				書き換え不可 (const)
				メソッド内から VERSION でアクセス可能
				継承クラスからアクセス可能
				外部から
					Vector::VERSION
					v.VERSION  (vが実体Vectorの場合)
					v->VERSION (vがポインタVector*の場合)
					でアクセス可能
			*/

	protected:
		// クラス変数
		static string description;
			/*
				クラス内で共有される値 (static)
				書き換え可能
				メソッド内から description でアクセス可能
				継承クラスからアクセス可能 (protected:)
				外部からはアクセス不可 (protected:)
			*/

	public:
		// インスタンス変数
		double x = 0;
		double y = 0;
		double z = 0;
		/*
			各インスタンス毎に異なる値
			Vector::でアクセスしたら0になる
			インスタンスメソッド内のみから x でアクセス可能
			publicなので,外部から v.x でアクセス可能
		*/

	public:

		// イニシャライザ/コンストラクタ
		Vector() {
			x = 0;
			y = 0;
			z = 0;
		};
		Vector(double X, double Y, double Z) {
			x = X;
			y = Y;
			z = Z;
		};

		// インスタンスの説明
		string desc() {
			char s[1000];
			sprintf(s,"(%d,%d,%d)",(int)x,(int)y,(int)z);
			return string(s);
		};

		// 足し算を定義

		// インスタンスメソッド版 (自分自身に足し合わせていく)
		void add(Vector v) {
			x += v.x;
			y += v.y;
			z += v.z;
		};

		template<typename... Vn>
		void add(Vector v1,Vn ...vn) {
			add(v1);
			add(vn...);
		};
			/*
				可変引数の add
				任意の個数のVectorを受け付ける。
				n個のVectorは Vector::add(Vector v1,Vn ...vn) に当てはめられ,型 Vn はn-1個のVectorの列になる。
				add(v1) で Vector::add(Vector v) が実行される。
				add(vn...) でn-1個のVectorである vn が展開されて Vector::add(Vector v1,Vn ...vn) が実行される。すると,Vnはn-2個のVectorの列になる。
				Vnが1個のVectorになると add(vn...) で Vector::add(Vector v) が実行される。
				これを繰り返して,1つずづVectorを処理する。
			*/

		// クラスメソッド版 (引数のVectorを足し合わせた結果を返す)
			// クラスメソッドでは static を前置する
		static Vector added(Vector v1, Vector v2) {
			return Vector(
				v1.x+v2.x,
				v1.y+v2.y,
				v1.z+v2.z
			);
		};
		template<typename... Vn>
		static Vector added(Vector v1, Vector v2, Vn ...vn) {
			return added(v1,added(v2,vn...));
		};
			/*
				addやaddedにおいて,
					void add(Vector v1,Vn ...vn);
					static Vector added(Vector v1, Vn ...vn);
				といった宣言だけで終わらせるのはよくない。なぜなら,これはテンプレートを含んでおり具体的な関数形がなく抽象的なので,.hppの宣言と.cppの実装を分けると.cppをコンパイルした時点ではオブジェクトファイルには実体がなく,リンクできないからである。
				リンカでは,要求に応じてオブジェクトファイルの中から具体的な Vector added(Vector,Vector...) を探すのだが,見つからないのでリンクに失敗する。
				具体的に Vector added(Vector,Vector,Vector) などと実装すると使えるのだが,それだと可変個引数に対応できない。よって,宣言時に実装するしかない。
			*/

		// スカラ倍を定義 (共にインスタンスメソッド)

		// 自分自身を実数倍
		void coefMultiply(double k) {
			x *= k;
			y *= k;
			z *= k;
		};

		// 自分自身の実数倍のVectorを生成
		Vector coefMultiplied(double k) {
			return Vector(x*k,y*k,z*k);
		};

		// クラス変数 description を返す
		static string describe() {
			return description;
		};

};

/*
	public:
		このキーワード以後に書かれた関数やデータは,外部からアクセス可能
	private:
		このキーワード以後に書かれた関数やデータは,このクラス内でのみアクセス可能
	protected:
		このキーワード以後に書かれた関数やデータは,このクラスと,継承したクラス内でのみアクセス可能
*/

本来はプロパティ `VERSION` や `description` を定義する次の行がクラス宣言の外に必要である。
```C++
const string Vector::VERSION = "1.0";
string Vector::description = "C++ simple vector class";
```
しかし, cling の実行が関数内で行われるせいか,こうした宣言ができない。

## クラス定義2 (ExtendedVector inherits from Vector)

In [3]:
class ExtendedVector: public Vector { // 継承宣言
	public:
		ExtendedVector() : Vector() {};
		ExtendedVector(double X, double Y, double Z) : Vector(X,Y,Z) {};
			// 親クラス(Vector)のイニシャライザに初期化を代行させる
	public:
		// 内積を定義
		double dot(Vector v) {
			double p=0;
			p+=x*v.x;
			p+=y*v.y;
			p+=z*v.z;
			return p;
		};
		// 外積を定義
		Vector cross(Vector v) {
			return Vector(
				y*v.z-z*v.y,
				z*v.x-x*v.z,
				x*v.y-y*v.x
			);
		};
		// ノルムを定義
		double norm() {
			return sqrt(dot(*this));
		};
		// 説明できるはず
		static string describeFromSub() {
			return description;
		};
	private:
		// プライベートメソッド (外部からアクセスできない)
		// 単位ベクトルに変換
		void normalize() {
			this->coefMultiply(1/this->norm());
		};
};

## クラスの利用

In [4]:
Vector vec1(3,2,1);
ExtendedVector vec2(6,4,2);
ExtendedVector vec3(54,63,72);
Vector vec4;

// 値の設定
vec3.x = 16;

(double) 16.000000


In [5]:
cout <<
// オブジェクトそのものの説明といえば,ポインタしかない
"vec1: " << &vec1 << endl <<
"vec2: " << &vec2 << endl <<

endl <<

"vec1の説明: " << vec1.desc() << endl <<
"vec2の説明: " << vec2.desc() << endl <<
"vec3の説明: " << vec3.desc() << endl <<
"vec4の説明: " << vec4.desc() << endl <<

endl <<

"vec1のx座標: " << vec1.x << endl <<
"vec2のy座標: " << vec2.y << endl <<
"vec3のz座標: " << vec3.z << endl <<

endl <<

"vec1+vec2+vec3: " << Vector::added(vec1,vec2,vec3).desc() << endl;
vec4.add(vec2);
cout <<
"vec4+vec2: " << vec4.desc() << endl <<
"vec2×12: " << vec2.coefMultiplied(12).desc() << endl <<

endl <<

"vec2∙vec3: " << vec2.dot(vec3) << endl <<
"vec3×vec2: " << vec3.cross(vec2).desc() << endl <<
"|vec3|:    " << vec3.norm() << endl;

vec1: 0x10a5cf0c8
vec2: 0x10a5cf0e0

vec1の説明: (3,2,1)
vec2の説明: (6,4,2)
vec3の説明: (16,63,72)
vec4の説明: (0,0,0)

vec1のx座標: 3
vec2のy座標: 4
vec3のz座標: 72

vec1+vec2+vec3: (25,69,75)
vec4+vec2: (6,4,2)
vec2×12: (72,48,24)

vec2∙vec3: 492
vec3×vec2: (-162,400,-314)
|vec3|:    97


(std::__1::basic_ostream &) @0x7fff9648d760
